#Environment Setting

In [0]:
#List of dependencies
import os
import shutil
import copy
import pandas as pd
import datetime
import re
import time
import json
import difflib
## mp4 to .wav converter
import ffmpeg

import os
import azure.cognitiveservices.speech as speechsdk

SPEECH_REGION = "xxxx"

SPEECH_KEY="xxxx"


#####################################################################################
############################## GLOBAL VARIABLE SETTING #############################
#####################################################################################
if not('IS_FIRST_NOTEBOOK_RUN' in locals()):
#     os.system("apt-get -f -y install xxxxx")
    IS_FIRST_NOTEBOOK_RUN = True

In [0]:
container_name="xxxx"
storage_name="xxxx"
blobkey="xxxx"
blobconf = "fs.azure.account.key."+storage_name+".blob.core.windows.net"

if not any(mount.mountPoint == "/mnt/"+storage_name+ "/" +container_name for mount in dbutils.fs.mounts()):
    print('nothing mounted')
    try:
        dbutils.fs.mount(
          source = "wasbs://"+container_name+"@"+storage_name+".blob.core.windows.net",
          mount_point = "/mnt/"+storage_name+ "/" +container_name,
          extra_configs = {blobconf:blobkey})
    except Exception as e:
        print("already mounted. Try to unmount first")

PATH = '/dbfs/mnt/'+storage_name+'/'+container_name +'/'


#m4a to wav

In [0]:
!apt install -y ffmpeg





ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 137 not upgraded.


## Detect updated files

#Creat Uploaded File Time

In [0]:
from datetime import datetime
from datetime import timedelta, timezone

df_file_time = pd.DataFrame(columns = ['FileName','FileTime'])
for (dirpath, dirnames, filenames) in os.walk(PATH+"audio-file-input"+"/"):
    for filename in filenames:
        if filename.split('.')[1] not in ['csv','txt']:
            filepath = dirpath + filename
            filetime = os.path.getctime(filepath)
            
filename_list = []
filetime_list = []
TAT_list = []
now = datetime.now()
for (dirpath, dirnames, filenames) in os.walk(PATH+"audio-file-input"+"/"):
    for filename in filenames:
        if filename.split('.')[1] not in ['csv','txt']:
            filename_list.append(filename.split(".")[0])
            filepath = dirpath + filename
            filetime = os.path.getctime(filepath)
            filetime = datetime.fromtimestamp(filetime)
            # Create the UTC+7 timezone
            utc_plus_7 = timezone(timedelta(hours=7))
            filetime = filetime.astimezone(utc_plus_7)
            time_gap = now.timetuple().tm_yday - filetime.timetuple().tm_yday
            filetime = filetime.strftime("%Y-%m-%d %H:%M:%S")
            filetime_list.append(filetime)
            TAT_list.append(time_gap)
            
df_file_time =  pd.DataFrame({'FileName':filename_list,'FileTime':filetime_list, 'TAT':TAT_list})
df_file_time          

##Creat file info (timestamp)

In [0]:
from pydub import AudioSegment
import argparse

#formats_to_convert = ['.m4a']
filename = dbutils.widgets.get('filename')
filename = filename.split("/")[1] #define the filename ###.aac. m4a ...
if filename.split(".")[0] not in ['csv','txt']:

    #if filename.endswith(tuple(formats_to_convert)):
    filepath = PATH+"audio-file-input"+"/" + filename
    (path, file_extension) = os.path.splitext(filepath)
    file_extension_final = file_extension.replace('.', '')
    try:
        if file_extension_final == 'opus':
            track = AudioSegment.from_file(filepath, format="ogg", codec="libopus")
        else:
            track = AudioSegment.from_file(filepath, file_extension_final)
        wav_filename = filename.replace(file_extension_final, 'wav')
        wav_path = PATH + "wav-file-convert" + '/' + wav_filename
        file_handle = track.export(wav_path, format='wav')
        print('CONVERTING: ' + str(filepath))
        while not os.path.exists(wav_path):
            print(wav_path)
            file_handle = track.export(wav_path, format='wav')
    except:
        print("ERROR CONVERTING " + str(filepath))
else:
    pass

#Speech_to_text

##speech_language_detection_once_from_continuous

In [0]:
import time
import json
def recognize_from_file(FilePath):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    #speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)
    speech_config = speechsdk.SpeechConfig(endpoint = "xxxx", subscription = SPEECH_KEY)
    speech_config.speech_recognition_language="id-ID"
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>

    audio_config = speechsdk.audio.AudioConfig(filename=FilePath)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    phrase_list_grammar = speechsdk.PhraseListGrammar.from_recognizer(speech_recognizer)
    list_of_mandatory_phrases = [
        "xxxx"]
    
    for phrase in list_of_mandatory_phrases:
        phrase_list_grammar.addPhrase(phrase)
    

    done = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True
    
    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)
        
    speech_recognizer.recognized.connect(handle_final_result)
    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
    #Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()
    # </SpeechContinuousRecognitionWithFile>
    
    all_results = ' '.join(all_results)

    return all_results


In [0]:
# filename == ###.aac.m4a ...
a=''
for i in range(4):
    try:
        if filename.split(".")[1] not in ['csv','txt']:
            wav_file = filename.split(".")[0]+".wav"
            output = recognize_from_file(PATH+"wav-file-convert"+"/"+wav_file)
            with open(PATH+"text-file"+"/"+wav_file[:-4]+".txt", 'w') as f: #overwrite the file if it existing
                if output == 0:
                    output= ""
                f.write(output)
    except:
        a = 'failed'
    if a != 'failed':
        print(i)
        break

#File Created

In [0]:


text_folders = os.listdir(PATH+"text-file")
speech_list = []
speech_name = []

for file in text_folders:
    split_tup = os.path.splitext(file)
    if split_tup[1] == ".txt":
        with open(PATH+"text-file/"+file) as f:
            speech = f.read()
        speech_list.append(speech)
        speech_name.append(split_tup[0])

df = pd.DataFrame({'FileName':speech_name,
                  'Text':speech_list})       
df

#Differetiate Individuals & Company

In [0]:
def diff(text):
    text = text.replace(",","").replace(".", "").lower()
    text = text.replace('resiko','risiko')
    num =  [x.replace(" ","") for x in re.compile(r'\d[\d ?]{1,32}\d', re.M).findall(text[:360])]
    for i in num:
        text = text.replace(i,i.replace(" ",""))
    
    if len(num) == 1:
        diff = "Company"       
    elif len(num) == 2:
        diff = 'Individual'
    else:
        if 'nik' in text:

            diff = "Individual"
        else:
            diff = "Company"
    
    return diff
                    
df['Category'] = df['Text'].apply(diff)

df_company = df[df['Category'] == "Company"].reset_index(drop=True)
df_individual = df[df['Category'] == "Individual"].reset_index(drop=True)
df_company
 

#Template__text_to_find_for_company/individual

#Stemmer & Individual_Match

In [0]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
Fact = StemmerFactory()
Stemmer = Fact.create_stemmer()


text_to_find_for_individual_list =['"xxxx"] 

text_to_find_for_individual_list = [Stemmer.stem(i.replace(","," ").replace("."," ").lower()) for i in text_to_find_for_individual_list]

from nlp_id.lemmatizer import Lemmatizer 
lemmatizer = Lemmatizer() 
text_to_find_for_individual_list = [lemmatizer.lemmatize(i) for i in text_to_find_for_individual_list]


##Individual_Match

In [0]:
def indi_test(text):
    text = Stemmer.stem(text.replace(","," ").replace(".", " ").lower())
    text = text.replace('resiko','risiko')
    num = re.compile(r'\d[\d ?]{1,32}\d', re.M).findall(text)[0].replace(" ","")
    text = text.replace(re.compile(r'\d[\d ?]{1,32}\d', re.M).findall(text)[0],num)

    sent_0 = False
    sent_1 = False
    sent_2 = False
    sent_3 = False
    sent_4 = False
    sent_5 = False  
    
    
    if 'saya' in text.split()[:text.split().index(num)]:
        sent_0 = 'Perfect Match'
    else:
        sent_0 = False
    if len(re.findall(r"xxxx",text)) > 0: 
        if str(re.findall(r"xxxx",text)[0]) in text:
            sent_1 = True
            try:
                if str(re.findall(r"xxxx",text)[0]) == text_to_find_for_individual_list[1]:
                    sent_1 = 'Perfect Match'
            except:
                pass
    else:
        sent_1 = False
    if len(re.findall(r"xxxx",text))>0:
        if str(re.findall(r"xxxx"',text)[0]) in text:
            sent_2 = True
            if str(re.findall(r"xxxx",text)[0]) == text_to_find_for_individual_list[2]:
                sent_2 = 'Perfect Match'
    else:
        sent_2 = False
    if len(re.findall(r"xxxx",text))>0:
        if str(re.findall(r"xxxx",text)[0]) in text:
            sent_3 = True
            if str(re.findall(r"xxxx",text)[0]) == text_to_find_for_individual_list[3]:
                sent_3 = 'Perfect Match'
    else:
        sent_3 = False
    if len(re.findall(r"xxxx",text))>0:
        if str(re.findall(r"xxxx",text)[0]) in text:
            sent_4 = True
            try:
                if str(re.findall(r"xxxx",text)[0]) == text_to_find_for_individual_list[4]:
                    sent_4 = 'Perfect Match'
            except:
                pass
    else:
        sent_4 = False
    if len(re.findall(r"xxxx"',text))>0:
        if str(re.findall(r"xxxx",text)[0]) in text:
            sent_5 = True
            try:
                if str(re.findall(r"xxxx",text)[0]) == text_to_find_for_individual_list[5]:
                    sent_5 = 'Perfect Match'
            except:
                pass
    else:
        sent_5 = False
    
    
    sent = {'sent0':sent_0,
            'sent1':sent_1,
            'sent2':sent_2,
            'sent3':sent_3,
            'sent4':sent_4,
            'sent5':sent_5}
    
    Status = "Refuse" if False in sent.values() else("Accept" if True in sent.values() else "Perfect Match")
    sent = {'sent0':sent_0,
            'sent1':sent_1,
            'sent2':sent_2,
            'sent3':sent_3,
            'sent4':sent_4,
            'sent5':sent_5,
            'text_status':Status}
    return sent
        
#pd.DataFrame([indi_test(df.iloc[1]['Text'])]) 

## Individual File

In [0]:

df_temp_indi_1= pd.DataFrame()
df_temp_indi_2= pd.DataFrame()
for i in df_individual['Text'].apply(indi_Info_extract):
    df_temp = pd.DataFrame([i])
    df_temp_indi_1 = df_temp_indi_1.append(df_temp,ignore_index=True)

    
for i in df_individual['Text'].apply(indi_test):
    df_temp = pd.DataFrame([i])
    df_temp_indi_2 = df_temp_indi_2.append(df_temp,ignore_index=True)


df_individual = pd.concat([df_individual,df_temp_indi_1,df_temp_indi_2],axis=1)
df_individual = df_individual.merge(df_file_time,on='FileName',how='left')


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1813405632863760> in <cell line: 14>()
     12 
     13 df_individual = pd.concat([df_individual,df_temp_indi_1,df_temp_indi_2],axis=1)
---> 14 df_individual = df_individual.merge(df_file_time,on='FileName',how='left')

NameError: name 'df_file_time' is not defined

#Stemmer & Company_Match

In [0]:
text_to_find_for_company_list = [
"xxxx"
]
text_to_find_for_company_list = [Stemmer.stem(i.replace(",",' ').replace("."," ").lower()) for i in text_to_find_for_company_list]

from nlp_id.lemmatizer import Lemmatizer 
lemmatizer = Lemmatizer() 
text_to_find_for_company_list = [lemmatizer.lemmatize(i) for i in text_to_find_for_company_list]


##Company Match

In [0]:
def com_test(text):
    text = Stemmer.stem(text.replace(","," ").replace(".", " ").lower())
    text = text.replace('resiko','risiko')

    try:
        agent_code = re.compile(r'\d[\d ?]{1,32}\d', re.M).findall(text)[0].replace(" ","")
    except:
        pass
    try:
        text = text.replace(re.compile(r'\d[\d ?]{1,32}\d', re.M).findall(text)[0],agent_code)
    except:
        pass
    sent_0 = False
    sent_1 = False
    sent_2 = False
    sent_3 = False
    sent_4 = False
    sent_5 = False
    sent_6 = False 

    if 'saya' in text.split()[:3]:
        sent_0 = 'Perfect Match'
    else:
        sent_0 = False
    
    if len(re.findall(r"xxxx",text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_1 = True
            if str(re.findall(r'"xxxx"',text)[0]) == text_to_find_for_company_list[1]:
                sent_1 = 'Perfect Match'
    else:
        sent_1 = False
    
    if len(re.findall(r'"xxxx"',text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_2 = True
            if str(re.findall(r'"xxxx"',text)[0]) == text_to_find_for_company_list[2]:
                sent_2 = 'Perfect Match'
    else:
        sent2 = False
    if len(re.findall(r'"xxxx"',text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_3 = True
            if str(re.findall(r'"xxxx"',text)[0]) == text_to_find_for_company_list[3]:
                sent_3 = 'Perfect Match'
    else:
        sent_3 = False
    if len(re.findall(r'"xxxx"',text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_4 = True
            if str(re.findall(r'"xxxx"',text)[0]) == text_to_find_for_company_list[4]:
                sent_4 = 'Perfect Match'
    else:
        sent_4 = False
    if len(re.findall(r'"xxxx"',text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_5 = True
            if str(re.findall(r"xxxx"i',text)[0]) == text_to_find_for_company_list[5]:
                sent_5 = 'Perfect Match'
    else:
        sent_5 = False
    if len(re.findall(r'"xxxx"',text))>0:
        if str(re.findall(r'"xxxx"',text)[0]) in text:
            sent_6 = True
            if str(re.findall(r'"xxxx"',text)[0]) == text_to_find_for_company_list[6]:
                sent_6 = 'Perfect Match'
    else:
        sent_6 = False
        
    sent = {'sent0':sent_0,
            'sent1':sent_1,
            'sent2':sent_2,
            'sent3':sent_3,
            'sent4':sent_4,
            'sent5':sent_5,
            'sent6':sent_6}
    Status = "Refuse" if False in sent.values() else("Accept" if True in sent.values() else "Perfect Match")
    sent = {'sent0':sent_0,
            'sent1':sent_1,
            'sent2':sent_2,
            'sent3':sent_3,
            'sent4':sent_4,
            'sent5':sent_5,
            'sent6':sent_6,
            'text_status':Status}
    
    return sent
        


In [0]:
df_temp_comp_1= pd.DataFrame()
df_temp_comp_2= pd.DataFrame()
for i in df_company['Text'].apply(comp_Info_extract):
    df_temp = pd.DataFrame([i])
    df_temp_comp_1 = df_temp_comp_1.append(df_temp,ignore_index=True)
    
for i in df_company['Text'].apply(com_test):
    df_temp = pd.DataFrame([i])
    df_temp_comp_2 = df_temp_comp_2.append(df_temp,ignore_index=True)


df_company = pd.concat([df_company,df_temp_comp_1,df_temp_comp_2],axis=1)
df_company = df_company.merge(df_file_time,on='FileName',how='left')

#Match with .csv

In [0]:
directory = PATH+"audio-file-input"

df_list = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        #if filename != "file_info.csv":
        filepath = os.path.join(directory, filename)
        a = pd.read_csv(filepath)
        a['FileName'] = filename.split(".")[0]
        df_list.append(a)
df_csv = pd.concat(df_list, ignore_index=True)



In [0]:
df_company = df_company.merge(df_csv,on="FileName",how='left')
df_individual = df_individual.merge(df_csv,on="FileName",how="left")

#Data to DB

In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list 
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17
apt-get -y install unixodbc-dev
sudo apt-get install python3-pip -y
pip3 install --upgrade pyodbc

In [0]:
dbutils.fs.put("/databricks/init/<YourClusterName>/pyodbc-install.sh","""
#!/bin/bash
sudo apt-get update
sudo apt-get -q -y install unixodbc unixodbc-dev
sudo apt-get -q -y install python3-dev
/databricks/python/bin/pip install pyodbc
""", True)

import urllib

import pyodbc
for driver in pyodbc.drivers():
    print(driver)

from sqlalchemy import create_engine

server = "xxxx"
database = "xxxx"
username = "xxxx"
password = "xxxx"

driver = '{ODBC Driver 17 for SQL Server}'

odbc_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;UID='+username+';DATABASE='+ database + ';PWD='+ password
connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)
engine = create_engine(connect_str)

#used to create a table, will overwrite 

def to_sql(df, table):
    df.to_sql(table, engine, if_exists = "replace", index=False, chunksize = 100)

In [0]:
conn = pyodbc.connect(odbc_str)
sql = 'SELECT * FROM [dbo].["xxxx"]'
df = pd.read_sql(sql, conn)
duplicated_values = df.duplicated('FileName', keep=False)

# Drop the rows with duplicate values in 'col1' while keeping the latest occurrence
df = df[~duplicated_values]


df_result = pd.merge(df_result,df[['FileName','Manual Review']],on='FileName',how='left')
df_result.rename(columns = {'Manual Review_y':'Manual Review'}, inplace=True)
df_result.drop('Manual Review_x', inplace =True, axis=1)

filename = dbutils.widgets.get('filename')
filename = filename.split("/")[1]
# try:
#     df_result.loc[df_result['FileName'] == filename.split(".")[0], 'Manual Review'] = ''
# except:
#     pass
try:
    df_result.loc[df_result['FileName'].astype(int) == int(filename.split(".")[0]), 'Manual Review'] = ''
except:
    pass

df_result = df_result.drop_duplicates(subset='FileName', keep='last')


In [0]:
try: 
    to_sql(df_result,""xxxx"")
except: 
    to_sql(df_result,""xxxx"")

In [0]:
max_attempts = 2
success = False

for attempt in range(max_attempts):
    try:
        conn = pyodbc.connect(odbc_str)
        cur = conn.cursor()
        sql = '''ALTER TABLE "xxxx" ADD id INT NOT NULL PRIMARY KEY IDENTITY'''
        cur.execute(sql)
        conn.commit()
        success = True
        break
    except pyodbc.Error as e:
        if e.sqlstate == '23000':
            print(f"Error: Cannot create a non-unique primary key for the 'id' column. Retrying... (Attempt {attempt + 1})")
        else:
            print(f"An unexpected error occurred: {e}")
            conn.rollback()
    finally:
        cur.close()
        conn.close()

if not success:
    print("Failed to add 'id' as primary key after multiple attempts.")
